In [1]:
# CredPolScriptInteractive.py
import fire
import uuid
import textwrap
import sys
from llama_stack_client import LlamaStackClient, Agent, AgentEventLogger, RAGDocument
from llama_stack_client.types import Document, UserMessage
from termcolor import colored
from utils import check_model_is_available, get_any_available_model


In [2]:
# Crear el cliente
# Tesla T4
T4_llama_stack_endpoint = "http://20.96.106.121:5001"       

# H100
H100_llama_stack_endpoint = "http://20.246.72.227:5001"

llama_stack_endpoint=T4_llama_stack_endpoint
model_id = None

client = LlamaStackClient(base_url=llama_stack_endpoint)

In [3]:
# Crear una instancia de base de datos vectorial
vector_db_id = f"v{uuid.uuid4().hex}"
embed_lm = next(m for m in client.models.list() if m.model_type == "embedding")
embedding_model = embed_lm.identifier
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model=embedding_model,
    embedding_dimension=384,
    provider_id="faiss",
)

VectorDBRegisterResponse(embedding_dimension=384, embedding_model='all-MiniLM-L6-v2', identifier='v54a0cf79d8ec4094ae8055ac9e61b07b', provider_id='faiss', provider_resource_id='v54a0cf79d8ec4094ae8055ac9e61b07b', type='vector_db', access_attributes=None)

In [4]:
# Cargar información al RAG
urls = ["Politicas.txt"]
documents = [
    RAGDocument(
        document_id=f"num-{i}",
        content=f"https://raw.githubusercontent.com/pmontiveros/llama-stack-demo/refs/heads/main/data/Politicas/{url}",
        mime_type="text/plain",
        metadata={},
    )
    for i, url in enumerate(urls)
]

In [5]:
# Insertar documentos
client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=32,
)

In [6]:
# Verificación de escudos y selección del modelo
available_shields = [shield.identifier for shield in client.shields.list()]
if not available_shields:
    print(colored("No available shields. Disabling safety.", "yellow"))
else:
    print(f"Available shields found: {available_shields}")


if model_id is None:
    model_id = get_any_available_model(client)
    if model_id is None:
        sys.exit("No hay un modelo")
else:
    if not check_model_is_available(client, model_id):
        sys.exit("El modelo no está disponible")

print(f"Using model: {model_id}")

No available shields. Disabling safety.
Using model: meta-llama/Llama-3.2-3B-Instruct


In [15]:
# Custom Tool para identificar la intención del usuario
# ServerSide con docstring
def identificar_intencion(user_prompt: str) -> str:
    """
    Identifies the intent behind a user's prompt for a credit assistant by querying the model.

    :param user_prompt: The user's input text.
    :return: A string representing the identified intent, one of 'Consulta', 'Solicitud', or 'Reclamo'.
    """
    global client
    global model_id
    prompt = textwrap.dedent(
        """
        Eres un clasificador de intenciones para un asistente crediticio. Tu tarea es analizar el siguiente texto del usuario y determinar si expresa alguna de las siguientes categorias:

        - "iniciar_modelado": el usuario quiere comenzar el desarrollo de un modelo de riesgo.
        - "explorar_datos": el usuario quiere analizar, entender o validar los datos de entrada.
        - "generar_codigo": el usuario solicita ayuda para escribir código SQL o Python.
        - "modificar_codigo": el usuario desea cambiar o ajustar código ya existente.
        - "ejecutar_codigo": el usuario quiere que se corra un análisis o script.
        - "ver_resultados": el usuario busca interpretar o visualizar los resultados de un análisis.
        - "consultar_metodologia": el usuario hace referencia a un estándar, paper, guía o documento metodológico.
        - "consulta_regulatoria": el usuario pregunta sobre cumplimiento normativo.
        - "otra": no puede clasificarse en ninguna de las anteriores.

        Ejemplos:
        - iniciar_modelado: "Quiero empezar a construir un modelo de PD para cartera comercial."
        - explorar_datos: "¿Cómo identifico las variables más importantes en el dataset?"
        - generar_codigo: "Escribime un código en Python que calcule tasas de default."
        - modificar_codigo: "¿Cómo ajusto el código para usar cuartiles en vez de deciles?"
        - ejecutar_codigo: "Quiero ejecutar el análisis de variables predictoras."
        - ver_resultados: "Mostrame los resultados del modelo y la importancia de variables."
        - consultar_metodologia: "Nuestra metodología indica usar validación cruzada. ¿Aplicamos?"
        - consulta_regulatoria: "¿Cumple esto con Basilea III?"
        - otra: "Contame un chiste."
        
        Devuelve SOLO la categoria que corresponda.

        Texto del usuario: "{}"

        Respuesta:
        """
    ).format(user_prompt)

    try:
        response = client.inference.chat_completion(
            messages=[
                UserMessage(
                    content=prompt,
                    role="user",
                ),
            ],
            model_id=model_id,
            stream=False,
        )
        
        intent = response.completion_message.content
        valid_intents = ["Consulta", "Solicitud", "Reclamo"]
      #  if intent not in valid_intents:
      #      return "Consulta"
        return intent
    except Exception as e:
        print(f"Error al clasificar la intención: {str(e)}")
        return "Error"


In [33]:
def iniciar_modelado(user_prompt: str) -> str:
    """
    Simula la respuesta inicial del asistente al comenzar el desarrollo de un modelo de riesgo de crédito.
    :param user_prompt: The user's input text.
    :return: Devuelve los pasos para inciar el modelado
    """
    return (
        "Perfecto. Para comenzar con el desarrollo del modelo de probabilidad de default (PD), "
        "debemos primero definir el objetivo del modelo, la población objetivo (universo de análisis), "
        "el horizonte temporal de observación, y los eventos de default que serán utilizados como variable dependiente. "
        "¿Contás con una base de datos histórica con etiquetas de default, o querés que trabajemos sobre una estructura estándar simulada?"
    )


In [34]:
def explorar_datos(user_prompt: str) -> str:
    """
    Simula una sugerencia de análisis exploratorio de datos (EDA).
    :param user_prompt: The user's input text.
    :return: Devuelve los pasos de análisis exploratorio
    """
    return (
        "Para explorar los datos, comenzaremos revisando el diccionario de variables, el porcentaje de valores nulos, "
        "las distribuciones de las variables numéricas y categóricas, y los niveles de correlación entre ellas. "
        "También es útil calcular tasas de default por segmento para detectar patrones tempranos. "
        "¿Querés que prepare una rutina de EDA o que revisemos juntos las variables más relevantes?"
    )


In [35]:
def generar_codigo(user_prompt: str) -> str:
    """
    Simula una respuesta del asistente generando código sugerido.
    :param user_prompt: The user's input text.
    :return: Devuelve código sugerido.
    """
    return (
        "Aquí tenés un ejemplo de código en Python para calcular la tasa de default por decil de una variable continua:\n\n"
        "```python\n"
        "import pandas as pd\n"
        "df['decil'] = pd.qcut(df['edad'], 10, labels=False)\n"
        "default_rates = df.groupby('decil')['default'].mean()\n"
        "print(default_rates)\n"
        "```\n"
        "¿Querés que lo adapte a otra variable o lo traduzca a SQL?"
    )


In [36]:
def modificar_codigo(user_prompt: str) -> str:
    """
    Simula la modificación de un fragmento de código ya existente.
    :param user_prompt: The user's input text.
    :return: Devuelve la modificación de un fragmento de código ya existente.
    """
    return (
        "Entendido. Si el código original agrupa por deciles, pero ahora querés trabajar con cuartiles, podés cambiar esta línea:\n\n"
        "`df['decil'] = pd.qcut(df['edad'], 10, labels=False)`\n\n"
        "por esta otra:\n\n"
        "`df['cuartil'] = pd.qcut(df['edad'], 4, labels=False)`\n\n"
        "¿Te gustaría que además lo grafiquemos?"
    )

In [37]:
def ejecutar_codigo(user_prompt: str) -> str:
    """
    Simula la ejecución de un análisis y entrega un resumen de resultados.
    :param user_prompt: The user's input text.
    :return: Devuelve la ejecución de un análisis y entrega un resumen de resultados.
    """
    return (
        "Ejecuté el análisis solicitado. Aquí están los resultados:\n\n"
        "- Variables con mayor poder predictivo: `edad`, `ingresos_mensuales`, `mora_12m`.\n"
        "- La AUC preliminar del modelo base es de 0.72.\n"
        "- Las tasas de default aumentan consistentemente con la variable `mora_12m`.\n\n"
        "¿Querés avanzar con un modelo logístico o explorar otra técnica?"
    )

In [38]:
def ver_resultados(user_prompt: str) -> str:
    """
    Simula la presentación de resultados de un modelo de riesgo.
    :param user_prompt: The user's input text.
    :return: Devuelve un texto con la presentación de resultados de un modelo de riesgo.
    """
    return (
        "Aquí está el resumen de resultados del modelo final:\n\n"
        "- AUC (train): 0.78\n"
        "- AUC (test): 0.75\n"
        "- KS (test): 0.43\n"
        "- Variables más importantes: `mora_12m`, `ingresos_mensuales`, `score_interno`\n"
        "- Análisis de bondad de ajuste muestra buen calce en los deciles intermedios, pero cierta subestimación de riesgo en los extremos.\n\n"
        "¿Te gustaría generar un informe de validación o ajustar el modelo?"
    )

In [39]:
def consultar_metodologia(user_prompt: str) -> str:
    """
    Simula una respuesta del agente al interpretar una metodología.
    :param user_prompt: The user's input text.
    :return: Devuelve la interpretación de la metodología.
    """
    return (
        "Según la metodología proporcionada, se recomienda realizar segmentaciones previas a la modelización cuando "
        "existen grupos con comportamiento diferencial significativo. Además, se sugiere aplicar validaciones cruzadas "
        "y realizar un análisis de estabilidad por ventana temporal. ¿Querés que adaptemos el flujo de trabajo a estas pautas?"
    )


In [40]:
def consulta_regulatoria(user_prompt: str) -> str:
    """
    Simula una respuesta a una consulta normativa relacionada al modelado.
    :param user_prompt: The user's input text.
    :return: Devuelve una respuesta a una consulta normativa relacionada al modelado.
    """
    return (
        "Bajo las guías de Basilea II/III, los modelos de riesgo deben cumplir con requisitos de discriminación, calibración "
        "y estabilidad. Además, se exige documentación exhaustiva, revisión independiente y pruebas de backtesting. "
        "¿Querés que te ayude a mapear tu modelo contra estos criterios regulatorios?"
    )


In [28]:
p=identificar_intencion("¿con que requisitos regulatorios deben cumplir los modelos de riesgo?")
print(p)

La categoría corresponde a "consulta_regulatoria".


In [41]:
# Configuración del Agente
agent = Agent(
    client,
    model=model_id,
    instructions=textwrap.dedent(
        """
            Actúas como un experto en modelos de riesgo crediticio, especializado en el desarrollo de modelos de probabilidad de default (PD), pérdida dada default (LGD), y exposición al default (EAD). Estás asistiendo a analistas de riesgo en el diseño y documentación de un modelo de PD para una cartera de créditos comerciales.
            
            Tu objetivo es:
            - Guiar paso a paso el desarrollo del modelo, desde los datos hasta la validación.
            - Ayudar a interpretar y aplicar metodologías internas o regulaciones, si el usuario las provee.
            - Aportar buenas prácticas desde literatura o experiencia profesional.
            - Proponer análisis (EDA, univariados, multivariados, validaciones) y sugerir código (SQL, Python), solo cuando se solicite o sea útil.
            - Invocar herramientas del sistema cuando sea necesario para completar una tarea o extraer información clave.
            
            No asumas que hay datos reales conectados, salvo que el usuario provea entradas explícitas. Cuando recibas documentos o datasets, procesalos usando las herramientas provistas por el sistema.
            
            Sé claro, preciso y técnico en tus respuestas. Ante cualquier ambigüedad, solicitá más contexto antes de continuar.

            Lo primero que debes hacer es verificar la intención del usuario usando la función identificar_intencion.

            Y luego llamar a la función que corresponda a cada categoria.

        """
    ),
    tools=[
        {
            "name": "builtin::rag/knowledge_search",
            "args": {"vector_db_ids": [vector_db_id]},
        },
        identificar_intencion,
        iniciar_modelado,
        explorar_datos,
        generar_codigo,
        modificar_codigo,
        ejecutar_codigo,
        ver_resultados,
        consultar_metodologia,
        consulta_regulatoria
    ],
    input_shields=available_shields,
    output_shields=available_shields,
    enable_session_persistence=False,
)


In [42]:
# Creación de la sesión
session_id = agent.create_session(session_name=f"s{uuid.uuid4().hex}")

In [43]:
# Mock user_inputs
user_inputs = [
    "Estoy empezando con el desarrollo de un modelo de probabilidad de default para cartera comercial. ¿Qué lineamientos regulatorios tengo que tener en cuenta?",
    "Sí, por favor. También quiero usar como base la metodología de Credit Scoring de Thomas, ¿podés resumirme sus pautas principales?",
    "Sí, estoy usando una base de datos histórica. ¿Podés ayudarme a entenderla y prepararla?",
    "Comenzar con un análisis exploratorio automático en la base de datos",
    "Sí. Y después quiero correr un análisis univariado para ver qué variables están mejor asociadas con el default. ",
    "Ahora Necesito validaciones. ¿Podés calcular Gini y KS para este modelo?",
    "Bien, con los resultados prepara un informe completo con tablas, gráficos y código comentado"
]

In [49]:
# Running the turns
for t in user_inputs:
    print("user>", t)
    turn_response = agent.create_turn(
        messages=[{"role": "user", "content": t}], session_id=session_id, stream=True
    )
    for event in AgentEventLogger().log(turn_response):
        event.print()


user> Estoy empezando con el desarrollo de un modelo de probabilidad de default para cartera comercial. ¿Qué lineamientos regulatorios tengo que tener en cuenta?


APITimeoutError: Request timed out.

In [47]:
from IPython.display import display, Markdown
#from llama_index.core.agent.types import AgentChatResponse

# Running the turns
for t in user_inputs:
    display(Markdown("user >", t))

    # Ejecutar el turno con el agente (streaming)
    turn_response = agent.create_turn(
        messages=[{"role": "user", "content": t}],
        session_id=session_id,
        stream=True
    )

    # Capturar y juntar el texto del asistente
    full_response = ""
    for event in AgentEventLogger().log(turn_response):
        if hasattr(event, "text") and event.text:
            full_response += event.text

    # Mostrar respuesta formateada
    display(Markdown(f"**asistente >**\n\n{full_response}"))

user > Estoy empezando con el desarrollo de un modelo de probabilidad de default para cartera comercial. ¿Qué lineamientos regulatorios tengo que tener en cuenta?


**asistente >**



user > Sí, por favor. También quiero usar como base la metodología de Credit Scoring de Thomas, ¿podés resumirme sus pautas principales?


**asistente >**



user > Sí, estoy usando una base de datos histórica. ¿Podés ayudarme a entenderla y prepararla?


...

**asistente >**



user > Ahora Necesito validaciones. ¿Podés calcular Gini y KS para este modelo?


**asistente >**



user > Bien, con los resultados prepara un informe completo con tablas, gráficos y código comentado


**asistente >**



In [50]:
p=1+1